In [61]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split

categoryDF=pd.read_csv("Z:/MSBooks/NLP/Project/aspectCategoryDetection.csv",delimiter='#',encoding = 'utf-8')
del categoryDF['aspectTermPolarity']

In [62]:
domain=[]
for id in categoryDF.index:
    domain.append(categoryDF.iloc[id,0][:3])
categoryDF['domain'] = domain
categoryDF.head()

,domainID,aspectCategoryPolarity,review,aspectCategory,domain
0,app_2,neu,फेसबुक का सिक्योरिटी चैकअप फीचर पॉपअप की तरह य...,gui,app
1,app_3,neu,"इस पॉपअप बॉक्स में पासवर्ड चेंज करने, लॉगइन अल...",misc,app
2,app_4,neu,आप यहीं से पुराने लॉगइन सेशंस को हमेशा के लिए ...,gui,app
3,app_5,neu,ऐसे में कई बार ऐसा होता है कि आपका लॉगइन ओपन ह...,NaN,app
4,app_6,pos,अब नए फीचर से इस तरह की गलतियां नहीं होंगी और ...,NaN,app


In [63]:
categorySizeDF = pd.DataFrame({"count":categoryDF.groupby('domain',sort='False').size()})
categorySizeDF = categorySizeDF.reset_index()
categorySizeDF

,domain,count
0,app,229
1,cam,150
2,hea,43
3,hom,84
4,lap,348
5,mob,1141
6,mov,890
7,sma,330
8,spe,47
9,tab,1244


In [64]:
temp = categorySizeDF.ix[10]
categorySizeDF.ix[10] = categorySizeDF.ix[11]
categorySizeDF.ix[11]= temp

In [65]:
categorySizeDF

,domain,count
0,app,229
1,cam,150
2,hea,43
3,hom,84
4,lap,348
5,mob,1141
6,mov,890
7,sma,330
8,spe,47
9,tab,1244


In [66]:
dfList=[]
currentIndex= 0 
for id in categorySizeDF.index:    
    if id == 0:
        currentIndex=categorySizeDF.iloc[id,1]
        dfList.append(categoryDF[:currentIndex])
        
    else:
        nextIndex= currentIndex  + categorySizeDF.iloc[id,1]
        dfList.append(categoryDF[currentIndex:nextIndex])
        currentIndex=nextIndex



In [67]:
entertainmentDF = dfList[6].append(dfList[10])
electronicDF = dfList[1]
for i in range(len(dfList)):
    if i>1 and i!=6 and i!=10:
        electronicDF = electronicDF.append(dfList[i])

entertainmentDF = entertainmentDF.dropna(axis=0,how='any')
electronicDF = electronicDF.dropna(axis=0,how='any')

In [68]:
entertainmentDF['review'].to_csv("Z:/MSBooks/NLP/Project/categorysets/eReview.txt",encoding='utf-8', index= False)
electronicDF['review'].to_csv("Z:/MSBooks/NLP/Project/categorysets/elecReview.txt",encoding='utf-8',index = False)

In [69]:
import codecs
import hindiTokenizer
from hindiTokenizer import Tokenizer
import nltk


with codecs.open("Z:/MSBooks/NLP/Project/categorysets/eReview.txt",encoding='utf-8') as eReview:
    eReviewLines = eReview.readlines()

with codecs.open("Z:/MSBooks/NLP/Project/categorysets/elecReview.txt",encoding='utf-8') as elecReview:
    elecReviewLines = elecReview.readlines()


In [70]:
import codecs
reviewList=[]
posReview = codecs.open('Z:/MSBooks/NLP/Project/postaggedreviews.txt',encoding='utf-8')
reviewList=[x.strip() for x in posReview.readlines()]

In [71]:
print reviewList[1]

app_3/QC इस/DEM पॉपअप/NN बॉक्स/NN में/PSP पासवर्ड/JJ चेंज/NN करने/VM ,/SYM लॉगइन/NN अलर्ट्स/NNP चालू/NN करने/VM और/CC मौजूदा/JJ फेसबुक/NN सेशन/NN को/PSP पूरी/JJ तरह/NN से/PSP बंद/JJ करने/VM के/PSP ऑप्शन/NN मिलेंगे/VM ./SYM


In [72]:
from collections import OrderedDict
adjectives=OrderedDict()
for item in reviewList:
    idAndTagged = item.split()
    id = idAndTagged[0]
    taggedSentence = idAndTagged[1:]
    domain = id.split("/")[0]
    definingTerms = [s for s in taggedSentence if "JJ" in s]
    adjTerms = []
    for items in definingTerms:
        word=items.split("/")[0]
        word = word.encode('utf-8')
        adjTerms.append(word)
        adjectives[domain]=adjTerms




In [73]:
adjectiveDF = pd.DataFrame.from_dict(adjectives,orient='index')



adjectiveDF = adjectiveDF.reset_index()
adjectiveDF = adjectiveDF.rename(columns={"index":"reviewID"})
adjectiveDF.head()



,reviewID,0,1,2,3,4,5,6,7
0,app_2,सिक्योरिटी,फीचर,None,None,None,None,None,None
1,app_3,पासवर्ड,मौजूदा,पूरी,बंद,None,None,None,None
2,app_4,पुराने,None,None,None,None,None,None,None
3,app_6,नए,मुश्किल,None,None,None,None,None,None
4,app_9,मुताबिक,कोर,None,None,None,None,None,None


In [74]:
domain=[]
for id in adjectiveDF.index:
    domain.append(adjectiveDF.iloc[id,0][:3])
adjectiveDF['domain'] = domain

adjectiveDF

,reviewID,0,1,2,3,4,5,6,7,domain
0,app_2,सिक्योरिटी,फीचर,None,None,None,None,None,None,app
1,app_3,पासवर्ड,मौजूदा,पूरी,बंद,None,None,None,None,app
2,app_4,पुराने,None,None,None,None,None,None,None,app
3,app_6,नए,मुश्किल,None,None,None,None,None,None,app
4,app_9,मुताबिक,कोर,None,None,None,None,None,None,app
5,app_10,यूनिक,None,None,None,None,None,None,None,app
6,app_11,इन्स्पायर्ड,मॉडर्न,None,None,None,None,None,None,app
7,app_12,अडिशनल,None,None,None,None,None,None,None,app
8,app_13,खास,None,None,None,None,None,None,None,app
9,app_14,तहत,None,None,None,None,None,None,None,app


In [75]:
entAdjectives = entertainmentDF.merge(adjectiveDF,how='inner',left_on='domainID',right_on='reviewID')
elecAdjectives = electronicDF.merge(adjectiveDF,how='inner',left_on='domainID',right_on='reviewID')

In [76]:
entAdjectives = entAdjectives.drop(['domain_x','reviewID','domain_y'],axis=1)


In [77]:
elecAdjectives = elecAdjectives.drop(['domain_x','reviewID','domain_y'],axis=1)

In [78]:
entAdjectives.head(20)

,domainID,aspectCategoryPolarity,review,aspectCategory,0,1,2,3,4,5,6,7
0,mov_3,neu,जॉम्बी जोनर में बनने वाली फिल्म ग्लैमरस तो नही...,misc,बेस्ड,None,None,None,None,None,None,None
1,mov_9,con,फिल्म की कहानी अच्छी होते हुए भी उसे ठीक से पर...,story,अच्छी,None,None,None,None,None,None,None
2,mov_12,neg,बहुत ही डलनेस सा महसूस होता है।,misc,महसूस,None,None,None,None,None,None,None
3,mov_13,neu,अर्नाल्ड इस बार बहुत ही सीरियस किरदार में है।,performance,सीरियस,None,None,None,None,None,None,None
4,mov_16,con,अर्नाल्ड की मौजूदगी से फिल्म में कुछ देर के लि...,story,सही,None,None,None,None,None,None,None
5,mov_17,neu,फिल्म में कुछ देर बाद अन्य हॉरर फिल्मों जैसा ह...,misc,अन्य,महसूस,None,None,None,None,None,None
6,mov_20,neg,यही इस फिल्म की सबसे बड़ी कमजोरी है।,misc,बड़ी,None,None,None,None,None,None,None
7,mov_23,pos,"दोनों देशों के बीच किरदारों के मेल-मिलाप, हंसी...",misc,दोनों,हास्यास्पद,मौजूद,None,None,None,None,None
8,mov_25,con,"अफसोस कि 'वेलकम 2 कराची' का विचार तो नेक, कटाक...",misc,कमजोर,None,None,None,None,None,None,None
9,mov_28,neu,"अभी जो भूमिका जैकी भगनानी ने निभाई है, उसी भूम...",performance,जैकी,None,None,None,None,None,None,None


In [79]:
entLexicon={}
for id in entAdjectives.index:
    category = entAdjectives.iloc[id,3]
    adjectives = entAdjectives.iloc[id][4:]
    adjectives = adjectives.dropna()
    #print adjectives
    for items in adjectives:
        adjList=[]
        if category in entLexicon.keys():
            entLexicon[category].append(items)
        else:
            adjList.append(items)
            entLexicon[category]=adjList
            
elecLexicon={}
for id in elecAdjectives.index:
    category = elecAdjectives.iloc[id,3]
    adjectives = elecAdjectives.iloc[id][4:]
    adjectives = adjectives.dropna()
    for items in adjectives:
        adjList=[]
        if category in elecLexicon.keys():
            elecLexicon[category].append(items)
        else:
            adjList.append(items)
            elecLexicon[category] = adjList
    

In [80]:
for keys in elecLexicon.keys():
    print len(elecLexicon[keys])

301
629
2053
120
753
497


In [81]:
word = "खाना".decode('utf-8', 'ignore')

import pickle

word2Synset = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/WordSynsetDict.pk"))
synset2Onto = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetOnto.pk"))
synonyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetWords.pk"))
#synset2Gloss = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetGloss.pk"))
#synset2Hypernyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHypernym.pk"))
#synset2Hyponyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHyponym.pk"))
#synset2Hypernyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHypernym.pk"))


if word2Synset.has_key(word):
    synsets = word2Synset[word]
    for pos in synsets.keys():
        for synset in synsets[pos]:
            if synonyms.has_key(synset):
                print "\t\t\t Synonyms -->", synonyms[synset]
                
                
            


			 Synonyms --> {'1': [u'\u0916\u093e\u0926\u094d\u092f_\u0935\u0938\u094d\u0924\u0941', u'\u0916\u093e\u0926\u094d\u092f_\u092a\u0926\u093e\u0930\u094d\u0925', u'\u0916\u093e\u0926\u094d\u092f\u0935\u0938\u094d\u0924\u0941', u'\u0916\u093e\u0926\u094d\u092f\u092a\u0926\u093e\u0930\u094d\u0925', u'\u0906\u0939\u093e\u0930', u'\u0916\u093e\u0926\u094d\u092f', u'\u092d\u094b\u091c\u094d\u092f_\u092a\u0926\u093e\u0930\u094d\u0925', u'\u0916\u093e\u0926\u094d\u092f_\u0938\u093e\u092e\u0917\u094d\u0930\u0940', u'\u0916\u093e\u0926\u094d\u092f-\u0938\u093e\u092e\u0917\u094d\u0930\u0940', u'\u0906\u0939\u093e\u0930_\u092a\u0926\u093e\u0930\u094d\u0925', u'\u0905\u0928\u094d\u0928', u'\u0906\u0939\u0930', u'\u0916\u093e\u0928\u093e', u'\u092b\u0942\u0921', u'\u0930\u0938\u0926', u'\u0905\u0930\u094d\u0915', u'\u0907\u0930\u093e', u'\u0924\u0906\u092e']}
			 Synonyms --> {'1': [u'\u092d\u094b\u091c\u0928', u'\u0916\u093e\u0928\u093e', u'\u0906\u0939\u093e\u0930', u'\u0905\u0928\u094d\u0928', u

In [82]:
import pickle

word2Synset = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/WordSynsetDict.pk"))
synset2Onto = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetOnto.pk"))
synonyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetWords.pk"))
synset2Gloss = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetGloss.pk"))
synset2Hypernyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHypernym.pk"))
synset2Hyponyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHyponym.pk"))
synset2Hypernyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHypernym.pk"))

In [83]:
import pickle
word2Synset = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/WordSynsetDict.pk"))
#synset2Onto = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetOnto.pk"))
synonyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetWords.pk"))
def getSynonyms(word):
    """import pickle

    word2Synset = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/WordSynsetDict.pk"))
    synset2Onto = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetOnto.pk"))
    synonyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetWords.pk"))"""
    #synset2Gloss = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetGloss.pk"))
    #synset2Hypernyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHypernym.pk"))
    #synset2Hyponyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHyponym.pk"))
    #synset2Hypernyms = pickle.load(open("Z:/MSBooks/NLP/Project/hindi_wordnet_python/SynsetHypernym.pk"))

    synList=[]
    if word2Synset.has_key(word):
        synsets = word2Synset[word]
        for pos in synsets.keys():
            for synset in synsets[pos]:
                if synonyms.has_key(synset):
                    #print "\t\t\t Synonyms -->", synonyms[synset]
                    synDict = synonyms[synset]
                    synList.append(synDict)
    #print len(synList)
    return synList
"""word = "खाना".decode('utf-8', 'ignore')
synList = getSynonyms(word)
for key,value in synList[1].iteritems():
    for item in value:
        print item"""
        
for key,item in entLexicon.iteritems():
    for words in item:
        if isinstance(words,str):
            word = words.decode('utf-8','ignore')
            synList = getSynonyms(word)
            if len(synList)>0:
                for k,value in synList[0].iteritems():
                    for syns in value:
                        entLexicon[key].append(syns)
       
        """for k,value in synList[1].iteritems():
            for syns in value:
                lexicon[key].append(syns)"""
        
for key,item in elecLexicon.iteritems():
    for words in item:
        if isinstance(words,str):
            word = words.decode('utf-8','ignore')
            synList = getSynonyms(word)
            if len(synList)>0:
                for k,value in synList[0].iteritems():
                    for syns in value:
                        elecLexicon[key].append(syns)


In [84]:
for keys in elecLexicon.keys():
    print len(elecLexicon[keys])

2461
5439
14753
830
6839
3513


In [85]:
import json,ast
with open("Z:/MSBooks/NLP/Project/termextracted/cat_final.txt") as termFile:
    termData = json.load(termFile)
termData = ast.literal_eval(json.dumps(termData, ensure_ascii=False, encoding='utf8'))


In [86]:
from collections import OrderedDict
categoryDict=OrderedDict()
for key,items in termData.iteritems():
    for k,v in items.iteritems():
        if len(v)>0:
            for items in v:
                for lexKey,lexItems in entLexicon.iteritems():
                    if items.decode('utf-8','ignore') in lexItems:
                        categoryDict[key] = lexKey
                for lk,li in elecLexicon.iteritems():
                    if items.decode('utf-8','ignore') in li:
                        categoryDict[key] = lk

C:\Users\Abijith Sankar\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
C:\Users\Abijith Sankar\Anaconda2\lib\site-packages\ipykernel\__main__.py:11: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [87]:
categoryDict

OrderedDict([('mov_204', u'software'),
             ('mob_1393', u'software'),
             ('tab_641', u'software'),
             ('tab_765', u'software'),
             ('tab_760', u'software'),
             ('tab_110', u'hardware'),
             ('lap_131', u'software'),
             ('mob_1421', u'software'),
             ('mob_1328', u'software'),
             ('lap_138', u'software'),
             ('app_32', u'design'),
             ('app_81', u'design'),
             ('mob_658', u'software'),
             ('cam_89', u'software'),
             ('tab_14', u'software'),
             ('mob_655', u'design'),
             ('tab_507', u'software'),
             ('tra_576', u'design'),
             ('tra_326', u'software'),
             ('tab_619', u'software'),
             ('lap_36', u'software'),
             ('mob_951', u'software'),
             ('tab_949', u'software'),
             ('sma_283', u'software'),
             ('mob_829', u'design'),
             ('mob_827', u'software')

In [88]:
#predictedCategoryDF = pd.DataFrame.from_dict(categoryDict)
predictedCategoryDF = pd.DataFrame(categoryDict.items(), columns=['reviewID', 'predictedCategory'])
predictedCategoryDF.head()

,reviewID,predictedCategory
0,mov_204,software
1,mob_1393,software
2,tab_641,software
3,tab_765,software
4,tab_760,software


In [122]:
finalDF = categoryDF.merge(predictedCategoryDF,left_on="domainID",right_on="reviewID")
finalDF['predictedCategory'].unique()

array([u'misc', u'hardware', u'software', u'design', u'place',
       u'ease_of_use', u'performance', u'price', u'reachability'], dtype=object)

In [123]:
finalDF = finalDF.dropna()

In [132]:
finalDF.reset_index(drop='True')
len(finalDF['predictedCategory'])

476

In [133]:
from sklearn.metrics import f1_score
f1_score(finalDF['aspectCategory'], finalDF['predictedCategory'], average='micro')  

0.22058823529411764

In [134]:
from sklearn.metrics import accuracy_score
accuracy_score(finalDF['aspectCategory'], finalDF['predictedCategory'])  

0.22058823529411764

In [135]:
for key,value in entLexicon.iteritems():
    print (key,":")
    for items in value:
        print items
    print ("!!!!!!!!!!!!!!!!!!!!!!")

(u'scenery', ':')
अलग-अलग
जंगली
हिमालयन
लुप्तप्राय
प्राकृतिक
काला
अलग-अलग
जंगली
कस्तूरी
तेंदुए
हिमालयन
नष्ट
शाहबलूत
लाल-पीले
मनमोहक
खूबसूरत
हरी-भरी
सुहाना
ऊंची-ऊंची
प्राकृतिक
बाकी
घने
हरे
नीलकुरंजी
पूरी
नीला
महत्त्वपूर्ण
खूबसूरत
खूबसूरत
खूबसूरत
हपोली
खूबसूरत
पूरे
मनमोहक
सही
पौराणिक
भव्य
लोकप्रिय
सम्पूर्ण
विहंगम
उपयुक्त
साबित
पर्णपाती
कांटेदार
शामिल
दिलचस्प
अवशेष
प्राचीन
अच्छी
हरे-भरे
विविध
वन्य
सुंदर
वापस
सुंदर
मनमोहक
सम्मोहित
अच्छी
पुराने
विभिन्न
खुली
समुद्री
भव्य
अनोखा
उपयुक्त
स्थित
सेफ्टी
पिल्लसलुंकी
मुख्य
आनंददायक
प्राकृतिक
भरपूर
आदर्श
विहंगम
बिखरी
प्रसिद्ध
घने
प्राकृतिक
आकर्षक
घुमावदार
शांत
आदर्श
मीठे
रोमांचक
खूबसूरत
बेहतर
मनमोहक
रंगबिरंगे
खूब
प्राकृतिक
पूरी
दुर्लभ
साइबेरियन
मुख्य
प्राकृतिक
घने
नैसर्गिक
भीगी
प्रफुल्लित
सूखे
बलूत
स्थित
शांत
आकषर्क
रंग-बिरंगे
आकर्षित
महकती
खूबसूरत
मनोरम
लुभावनी
प्राकृतिक
प्राकृतिक
रोमांचित
सुंदर
प्राकृतिक
आकर्षित
प्राकृतिक
रोमांचित
मनोहारी
खूबसूरत
मग्नमुग्ध
स्थित
विभिन्न
रोमांचित
सही
नमकीन
सुंदर
खूबसूरत
शानदार
मुख्य
छोटे
रमण्ीय
खडे
विशाल
प्राकृतिक
व

In [136]:
with open("Z:/MSBooks/NLP/Project/termextracted/aspect_terms_sentiment_output.txt") as termSentiFile:
    termSentiData = json.load(termSentiFile)
termSentiData = ast.literal_eval(json.dumps(termSentiData, ensure_ascii=False, encoding='utf8'))

In [137]:
termSentiData

{'mov_391': {},
 'mov_393': {},
 'mov_204': {'\xe0\xa4\xad\xe0\xa5\x82\xe0\xa4\xae\xe0\xa4\xbf\xe0\xa4\x95\xe0\xa4\xbe': 1},
 'mov_397': {},
 'mov_398': {},
 'mob_1393': {},
 'mov_53': {},
 'tab_963': {},
 'tab_641': {'\xe0\xa4\xb0\xe0\xa5\x88\xe0\xa4\xae': 1},
 'tab_648': {'\xe0\xa4\xae\xe0\xa5\x87\xe0\xa4\xae\xe0\xa5\x8b\xe0\xa4\xb0\xe0\xa5\x80': 0},
 'lap_437': {},
 'hom_34': {},
 'tab_886': {},
 'hom_37': {},
 'tab_766': {},
 'tab_765': {},
 'hom_39': {},
 'tab_760': {'\xe0\xa4\x95\xe0\xa5\x80\xe0\xa4\xae\xe0\xa4\xa4': 1},
 'app_80': {},
 'tab_110': {},
 'lap_131': {},
 'mob_1421': {},
 'mob_1328': {'\xe0\xa4\xac\xe0\xa4\xa8\xe0\xa4\xbe\xe0\xa4\xb5\xe0\xa4\x9f': -1},
 'tab_118': {'\xe0\xa4\xac\xe0\xa5\x88\xe0\xa4\x9f\xe0\xa4\xb0\xe0\xa5\x80': 0},
 'lap_138': {},
 'app_38': {'\xe0\xa4\xa1\xe0\xa4\xbf\xe0\xa4\xb2\xe0\xa5\x80\xe0\xa4\xb5\xe0\xa4\xb0\xe0\xa5\x80': -1,
  '\xe0\xa4\xae\xe0\xa5\x88\xe0\xa4\xb8\xe0\xa5\x87\xe0\xa4\x9c': 1},
 'tra_474': {},
 'hom_50': {},
 'tel_135': {'\xe0

In [138]:
categorySentiScore={}
for key,values in termSentiData.iteritems():
    if len(values)>0:
        for k,v in values.iteritems():
            for entKey,entVal in entLexicon.iteritems():
                if k in entVal:
                    if entKey in categorySentiScore:
                        categorySentiScore[entKey] += v
                    else:
                        categorySentiScore[entKey] = v
            for elecKey,elecVal in elecLexicon.iteritems():
                if k in elecVal:
                    if elecKey in categorySentiScore:
                        categorySentiScore[elecKey] += v
                    else:
                        categorySentiScore[elecKey] = v
                    

C:\Users\Abijith Sankar\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
C:\Users\Abijith Sankar\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [145]:
predictedCategory = finalDF['predictedCategory']
predictedCategorySentiment=[]
for category in predictedCategory:
    if category in categorySentiScore.keys():
        if categorySentiScore[category] > 0:
            predictedCategorySentiment.append('pos')
        elif categorySentiScore[category] == 0:
            predictedCategorySentiment.append('neu')
        elif categorySentiScore[category] < 0:
            predictedCategorySentiment.append('neg')
    else:
        predictedCategorySentiment.append('neu')


In [150]:
finalDF['predictedCategorySentiment'] = predictedCategorySentiment
f1_score(finalDF['aspectCategoryPolarity'], finalDF['predictedCategorySentiment'], average='micro') 


0.45798319327731091